In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import requests
import json
import datetime

In [2]:
LINK = "https://portal.gupy.io"
nome_vaga = 'Cientista de Dados'

def vitrine_vagas_gupy(LINK, nome_vaga):

    print('Abrindo vitrine de vagas')

    driver = webdriver.Chrome()
    driver.get(LINK)
    driver.maximize_window()

    try:
        cuidado_golpes = driver.find_element(By.XPATH, '//*[@id="radix-0"]/div[2]/button')
        cuidado_golpes.click()
    except:
        pass
    search = driver.find_element(By.XPATH, '//*[@id="undefined-input"]')
    search.send_keys(nome_vaga)
    search.send_keys(Keys.ENTER)

    SCROLL_PAUSE_TIME = 0.5

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    print('Vitrine completa!')

    return driver.find_elements(By.XPATH, '//*[@id="main-content"]/ul/li/div')

def coleta_dados_basicos_vagas(vitrine_de_vagas):

    print('Coletando dados das vagas pela vitrine')

    lista_vagas = []

    for vaga in vitrine_de_vagas:
        dict_aux = {}
        try:
            dict_aux['link'] = vaga.find_element(By.TAG_NAME,'a').get_attribute('href')
        except:
            dict_aux['link'] = np.nan

        dict_aux['data_coleta'] = datetime.datetime.now().strftime('%Y-%m-%d')
        
        try:
            dict_aux['titulo_vaga'] = vaga.find_element(By.XPATH,'a/div/h3').get_attribute('innerHTML')
        except:
            dict_aux['titulo_vaga'] = np.nan
        try:
            dict_aux['localidade'] = vaga.find_element(By.XPATH,'//span[@data-testid="job-location"]').get_attribute('innerHTML')
        except:
            dict_aux['localidade'] = np.nan
        try:
            dict_aux['modalidade'] = vaga.find_element(By.XPATH,'a/div/div[2]/div[2]/span').get_attribute('innerHTML')
        except:
            dict_aux['modalidade'] = np.nan
        try:
            dict_aux['contrato'] = vaga.find_element(By.XPATH,'a/div/div[2]/div[3]/span').get_attribute('innerHTML')
        except:
            dict_aux['contrato'] = np.nan
        try:
            dict_aux['pcd'] = vaga.find_element(By.XPATH,'a/div/div[2]/div[4]/span').get_attribute('innerHTML')
        except:
            dict_aux['pcd'] = np.nan

        lista_vagas.append(dict_aux)
    
    print('Dados da vitrine coletados!')

    return pd.DataFrame(lista_vagas)

def coleta_info_vagas(links):

    print('Coletando informações das vagas individualmente')

    lista_vagas = []

    for link in links:

        try:

            dict_aux = {}
            driver = webdriver.Chrome()
            driver.get(link)
            driver.maximize_window()

            try:
                aceitar_cookies = driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
                aceitar_cookies.click()
            except:
                pass

            try:
                cuidado_golpes = driver.find_element(By.XPATH, '//*[@id="radix-0"]/div[2]/button')
                cuidado_golpes.click()
            except:
                pass

            try:

                json_vaga = driver.find_element(By.XPATH,'//*[@id="__NEXT_DATA__"]').get_attribute('innerHTML')

                json_vaga = json.loads(json_vaga)

                try:
                    dict_aux['link'] = link
                except:
                    dict_aux['link'] = np.nan   
                try:
                    dict_aux['nome_empresa'] = json_vaga['props']['pageProps']['job']['careerPage']['name']
                except:
                    dict_aux['nome_empresa'] = np.nan     
                try:
                    dict_aux['codigo_vaga'] = json_vaga['props']['pageProps']['job']['id']
                except:
                    dict_aux['codigo_vaga'] = np.nan
                try:
                    dict_aux['data_publicacao'] = json_vaga['props']['pageProps']['job']['publishedAt']
                except:
                    dict_aux['data_publicacao'] = np.nan
                try:
                    dict_aux['data_expiracao'] = json_vaga['props']['pageProps']['job']['expiresAt']
                except:
                    dict_aux['data_expiracao'] = np.nan
                try:
                    dict_aux['descricao'] = json_vaga['props']['pageProps']['job']['description']
                except:
                    dict_aux['descricao'] = np.nan
                try:
                    dict_aux['pre_requisitos'] = json_vaga['props']['pageProps']['job']['prerequisites']
                except:
                    dict_aux['pre_requisitos'] = np.nan
                try:
                    dict_aux['responsabilidades'] = json_vaga['props']['pageProps']['job']['responsibilities']
                except:
                    dict_aux['responsabilidades'] = np.nan
                try:
                    dict_aux['experiencias_relevantes'] = json_vaga['props']['pageProps']['job']['relevantExperiences']
                except:
                    dict_aux['experiencias_relevantes'] = np.nan
                try: 
                    dict_aux['cidade'] = json_vaga['props']['pageProps']['job']['addressCity']
                except:
                    dict_aux['cidade']
                try:
                    dict_aux['estado'] = json_vaga['props']['pageProps']['job']['addressState']
                except:
                    dict_aux['estado'] = np.nan

                lista_vagas.append(dict_aux)

            except:
                pass
        except:
            pass

        print(f'Progresso: {round(links.index(link)/len(links),2)*100} %')

        driver.quit()

    return pd.DataFrame(lista_vagas)

In [3]:
df_vagas_full = pd.DataFrame()

for nome_vaga in ['Cientista de Dados', 'Analista de Dados', 'Engenheiro de Dados']:

    vitrine_de_vagas = vitrine_vagas_gupy(LINK, nome_vaga)

    df_vagas1 = coleta_dados_basicos_vagas(vitrine_de_vagas)

    df_vagas2 = coleta_info_vagas(df_vagas1['link'].tolist())

    df_vagas = pd.merge(df_vagas1, df_vagas2, on='link', how='left')
    df_vagas['posicao'] = nome_vaga

    df_vagas_full = pd.concat([df_vagas_full, df_vagas], axis=0)

df_vagas_full = df_vagas_full.reset_index(drop=True)

df_vagas_full.to_excel('../data/data_raw/vagas_gupy_raw.xlsx', index = False)

Abrindo vitrine de vagas
Vitrine completa!
Coletando dados das vagas pela vitrine
Dados da vitrine coletados!
Coletando informações das vagas individualmente
Progresso: 0.0 %
Progresso: 1.0 %
Progresso: 3.0 %
Progresso: 4.0 %
Progresso: 5.0 %
Progresso: 7.000000000000001 %
Progresso: 8.0 %
Progresso: 10.0 %
Progresso: 11.0 %
Progresso: 12.0 %
Progresso: 14.000000000000002 %
Progresso: 15.0 %
Progresso: 16.0 %
Progresso: 18.0 %
Progresso: 19.0 %
Progresso: 21.0 %
Progresso: 22.0 %
Progresso: 23.0 %
Progresso: 25.0 %
Progresso: 26.0 %
Progresso: 27.0 %
Progresso: 28.999999999999996 %
Progresso: 30.0 %
Progresso: 32.0 %
Progresso: 33.0 %
Progresso: 34.0 %
Progresso: 36.0 %
Progresso: 37.0 %
Progresso: 38.0 %
Progresso: 40.0 %
Progresso: 41.0 %
Progresso: 42.0 %
Progresso: 44.0 %
Progresso: 45.0 %
Progresso: 47.0 %
Progresso: 48.0 %
Progresso: 49.0 %
Progresso: 51.0 %
Progresso: 52.0 %
Progresso: 53.0 %
Progresso: 55.00000000000001 %
Progresso: 56.00000000000001 %
Progresso: 57.99999999999

In [122]:
df_vagas_full.to_excel('../data/data_raw/vagas_gupy_raw.xlsx', index = False)

In [121]:
df_vagas_full.columns

Index(['link', 'titulo_da_vaga', 'localidade', 'modalidade', 'contrato', 'pcd',
       'codigo_vaga', 'data_publicacao', 'data_expiracao', 'descricao',
       'pre_requisitos', 'responsabilidades', 'experiencias_relevantes',
       'cidade', 'estado', 'posicao'],
      dtype='object')

In [119]:
df_vagas_full.columns = ['link', 'titulo_da_vaga', 'localidade', 'modalidade', 'contrato', 'pcd',
       'codigo_vaga', 'data_publicacao', 'data_expiracao', 'descricao',
       'pre_requisitos', 'responsabilidades', 'experiencias_relevantes',
       'cidade', 'estado', 'posicao']

In [84]:
def coleta_info_vagas(links):
    lista_vagas = []

    for link in links:

        print(link)

        dict_aux = {}
        driver = webdriver.Chrome()
        driver.get(link)
        driver.maximize_window()

        try:
            aceitar_cookies = driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
            aceitar_cookies.click()
        except:
            pass

        try:
            cuidado_golpes = driver.find_element(By.XPATH, '//*[@id="radix-0"]/div[2]/button')
            cuidado_golpes.click()
        except:
            pass

        try:

            json_vaga = driver.find_element(By.XPATH,'//*[@id="__NEXT_DATA__"]').get_attribute('innerHTML')

            json_vaga = json.loads(json_vaga)

            try:
                dict_aux['link'] = link
            except:
                dict_aux['link'] = np.nan        
            try:
                dict_aux['codigo_vaga'] = json_vaga['props']['pageProps']['job']['id']
            except:
                dict_aux['codigo_vaga'] = np.nan
            try:
                dict_aux['data_publicacao'] = json_vaga['props']['pageProps']['job']['publishedAt']
            except:
                dict_aux['data_publicacao'] = np.nan
            try:
                dict_aux['data_expiracao'] = json_vaga['props']['pageProps']['job']['expiresAt']
            except:
                dict_aux['data_expiracao'] = np.nan
            try:
                dict_aux['descricao'] = json_vaga['props']['pageProps']['job']['description']
            except:
                dict_aux['descricao'] = np.nan
            try:
                dict_aux['pre_requisitos'] = json_vaga['props']['pageProps']['job']['prerequisites']
            except:
                dict_aux['pre_requisitos'] = np.nan
            try:
                dict_aux['responsabilidades'] = json_vaga['props']['pageProps']['job']['responsibilities']
            except:
                dict_aux['responsabilidades'] = np.nan
            try:
                dict_aux['experiencias_relevantes'] = json_vaga['props']['pageProps']['job']['relevantExperiences']
            except:
                dict_aux['experiencias_relevantes'] = np.nan
            try: 
                dict_aux['cidade'] = json_vaga['props']['pageProps']['job']['addressCity']
            except:
                dict_aux['cidade']
            try:
                dict_aux['estado'] = json_vaga['props']['pageProps']['job']['addressState']
            except:
                dict_aux['estado'] = np.nan

            lista_vagas.append(dict_aux)

        except:
            pass

        driver.quit()

https://stefanini.gupy.io/job/eyJqb2JJZCI6Njk5NjY3Miwic291cmNlIjoiZ3VweV9wb3J0YWwifQ==?jobBoardSource=gupy_portal
https://bancosofisa.gupy.io/job/eyJqb2JJZCI6Njk4MzY2MCwic291cmNlIjoiZ3VweV9wb3J0YWwifQ==?jobBoardSource=gupy_portal
https://vemprabip.gupy.io/job/eyJqb2JJZCI6Njk4OTU5Nywic291cmNlIjoiZ3VweV9wb3J0YWwifQ==?jobBoardSource=gupy_portal
https://algartelecom.gupy.io/job/eyJqb2JJZCI6Njk2MTI0Mywic291cmNlIjoiZ3VweV9wb3J0YWwifQ==?jobBoardSource=gupy_portal
https://mutantbrvagas.gupy.io/job/eyJqb2JJZCI6Njk4NjY0Mywic291cmNlIjoiZ3VweV9wb3J0YWwifQ==?jobBoardSource=gupy_portal
https://sicoobunicentrobr.gupy.io/job/eyJqb2JJZCI6Njk3OTE1Mywic291cmNlIjoiZ3VweV9wb3J0YWwifQ==?jobBoardSource=gupy_portal
https://onset.gupy.io/job/eyJqb2JJZCI6Njk3MDcwOSwic291cmNlIjoiZ3VweV9wb3J0YWwifQ==?jobBoardSource=gupy_portal
https://bnex.gupy.io/job/eyJqb2JJZCI6Njk2MjAyOSwic291cmNlIjoiZ3VweV9wb3J0YWwifQ==?jobBoardSource=gupy_portal
https://bnex.gupy.io/job/eyJqb2JJZCI6Njk2MjExOSwic291cmNlIjoiZ3VweV9wb3J0YWwifQ=

In [85]:
df_vagas

,link,titulo_da_vaga,Localidade,Modalidade,Contrato,PDC
0,https://stefanini.gupy.io/job/eyJqb2JJZCI6Njk5...,Cientista de Dados PL/SR,Não informado,Remoto,Efetivo,Também p/ PcD
1,https://bancosofisa.gupy.io/job/eyJqb2JJZCI6Nj...,Cientista de Dados,Não informado,Híbrido,Efetivo,Também p/ PcD
2,https://vemprabip.gupy.io/job/eyJqb2JJZCI6Njk4...,Pessoa Cientista de Dados Pleno,Não informado,Remoto,Efetivo,Também p/ PcD
3,https://algartelecom.gupy.io/job/eyJqb2JJZCI6N...,Pessoa Cientista de Dados,Não informado,Híbrido,Associado,Também p/ PcD
4,https://mutantbrvagas.gupy.io/job/eyJqb2JJZCI6...,Cientista de Dados Sênior,Não informado,Híbrido,Efetivo,Também p/ PcD
...,...,...,...,...,...,...
67,https://a3data.gupy.io/job/eyJqb2JJZCI6NDgwNzU...,Banco de Talentos - Pessoa Cientista de Dados ...,Não informado,Remoto,Banco de Talentos,Também p/ PcD
68,https://slcmaquinas.gupy.io/job/eyJqb2JJZCI6ND...,Cientista de Dados Agro,Não informado,Presencial,Efetivo,Também p/ PcD
69,https://vagasewave.gupy.io/job/eyJqb2JJZCI6Mzk...,Cientista de Dados,Não informado,Remoto,Efetivo,NaN
70,https://dexco.gupy.io/job/eyJqb2JJZCI6MTYwNzA3...,Banco de Talentos - Cientista de Dados,Não informado,Remoto,Banco de Talentos,Também p/ PcD


In [98]:
round(df_info_vagas['link'].tolist().index('https://nuclea.gupy.io/job/eyJqb2JJZCI6Njk1MTA1OSwic291cmNlIjoiZ3VweV9wb3J0YWwifQ==?jobBoardSource=gupy_portal')/len(df_info_vagas['link'].tolist()),2)*100

14.000000000000002

In [86]:
df_info_vagas = pd.DataFrame(lista_vagas2)
df_info_vagas

,link,codigo_vaga,data_publicacao,data_expiracao,descricao,pre_requisitos,responsabilidades,experiencias_relevantes,cidade,estado
0,https://stefanini.gupy.io/job/eyJqb2JJZCI6Njk5...,6996672,2024-04-08T23:28:00.822Z,2024-04-30,"<p><span style=""color: black;"">📢 Oportunidade ...",<p><br></p><p><br></p>,"<p><span style=""color: black;"">🔎 Estamos procu...",<p><strong>O que te oferecemos:</strong></p><p...,,
1,https://bancosofisa.gupy.io/job/eyJqb2JJZCI6Nj...,6983660,2024-04-08T14:46:32.051Z,2024-05-10,<p><strong>Qual perfil buscamos?</strong></p><...,<ul><li>Experiência com base de dados</li><li>...,"<p><span style=""color: rgba(0, 0, 0, 0.9);"">Re...",<p>Será um diferencial:</p><p><br></p><ul><li>...,São Paulo,São Paulo
2,https://vemprabip.gupy.io/job/eyJqb2JJZCI6Njk4...,6989597,2024-04-08T13:53:32.256Z,2024-05-08,<p>Buscamos um cientista de dados com experiên...,<p><strong>Formação:</strong></p><ul><li>Super...,<p><strong>Principais atividades:</strong></p>...,<p><strong>CONHEÇA NOSSOS BENEFÍCIOS</strong><...,,
3,https://algartelecom.gupy.io/job/eyJqb2JJZCI6N...,6961243,2024-04-08T12:01:17.485Z,2024-04-08,"<p><span style=""color: rgb(0, 0, 0);"">Executar...",<ul><li>Sólidos conhecimentos de programação e...,<ul><li>Elabora metodologias de cálculo para a...,<p>Vale refeição</p><p>Assistência Médica</p><...,Uberlândia,Minas Gerais
4,https://mutantbrvagas.gupy.io/job/eyJqb2JJZCI6...,6986643,2024-04-05T17:52:22.645Z,2024-04-30,<p><strong>O QUE É SER UM MUTANT?</strong></p>...,<ul><li>Desenvolvimento em Python;</li><li>SQL...,<ul><li>Responsável pelo desenvolvimento de es...,<p><strong>O que você vai encontrar por aqui:<...,Osasco,São Paulo
...,...,...,...,...,...,...,...,...,...,...
65,https://bild.gupy.io/job/eyJqb2JJZCI6NTMzMzg0N...,5333845,2023-08-24T14:59:50.237Z,2023-09-11,"<p><strong style=""color: rgb(55, 65, 81);"">Sob...",<ul><li>Experiência sólida em desenvolvimento ...,<ul><li>Desenvolver soluções inovadoras em Pyt...,<ul><li>Faça parte de uma equipe dinâmica e co...,Ribeirão Preto,São Paulo
66,https://a3data.gupy.io/job/eyJqb2JJZCI6NDgwNzU...,4807510,2023-05-15T20:07:19.851Z,2023-12-31,"<p class=""ql-align-justify""><span style=""color...","<p class=""ql-align-justify""><strong style=""col...","<p>P<span style=""color: black;"">ara ser uma </...","<p class=""ql-align-justify""><strong style=""col...",Belo Horizonte,Minas Gerais
67,https://slcmaquinas.gupy.io/job/eyJqb2JJZCI6ND...,4456673,2023-03-15T17:16:23.137Z,2023-04-15,<p>Vem fazer parte de uma empresa Top Ser Huma...,"<ul><li class=""ql-align-justify""><span style=""...","<ul><li class=""ql-align-justify"">Coletar grand...",<p><strong>Benefícios:</strong></p><ul><li>Pla...,,None
68,https://dexco.gupy.io/job/eyJqb2JJZCI6MTYwNzA3...,1607079,2022-03-17T20:58:41.000Z,2023-12-31,"<p class=""ql-align-justify""><span style=""color...",<p>• Desenvolvimento e expansão de nosso DataL...,<p>• Atuar como cientis de dados na evolução e...,<p>• Conhecimentos específicos e obrigatórios:...,São Paulo,São Paulo


In [87]:
pd.merge(df_vagas, df_info_vagas, on = 'link', how = 'left')

,link,titulo_da_vaga,Localidade,Modalidade,Contrato,PDC,codigo_vaga,data_publicacao,data_expiracao,descricao,pre_requisitos,responsabilidades,experiencias_relevantes,cidade,estado
0,https://stefanini.gupy.io/job/eyJqb2JJZCI6Njk5...,Cientista de Dados PL/SR,Não informado,Remoto,Efetivo,Também p/ PcD,6996672.0,2024-04-08T23:28:00.822Z,2024-04-30,"<p><span style=""color: black;"">📢 Oportunidade ...",<p><br></p><p><br></p>,"<p><span style=""color: black;"">🔎 Estamos procu...",<p><strong>O que te oferecemos:</strong></p><p...,,
1,https://bancosofisa.gupy.io/job/eyJqb2JJZCI6Nj...,Cientista de Dados,Não informado,Híbrido,Efetivo,Também p/ PcD,6983660.0,2024-04-08T14:46:32.051Z,2024-05-10,<p><strong>Qual perfil buscamos?</strong></p><...,<ul><li>Experiência com base de dados</li><li>...,"<p><span style=""color: rgba(0, 0, 0, 0.9);"">Re...",<p>Será um diferencial:</p><p><br></p><ul><li>...,São Paulo,São Paulo
2,https://vemprabip.gupy.io/job/eyJqb2JJZCI6Njk4...,Pessoa Cientista de Dados Pleno,Não informado,Remoto,Efetivo,Também p/ PcD,6989597.0,2024-04-08T13:53:32.256Z,2024-05-08,<p>Buscamos um cientista de dados com experiên...,<p><strong>Formação:</strong></p><ul><li>Super...,<p><strong>Principais atividades:</strong></p>...,<p><strong>CONHEÇA NOSSOS BENEFÍCIOS</strong><...,,
3,https://algartelecom.gupy.io/job/eyJqb2JJZCI6N...,Pessoa Cientista de Dados,Não informado,Híbrido,Associado,Também p/ PcD,6961243.0,2024-04-08T12:01:17.485Z,2024-04-08,"<p><span style=""color: rgb(0, 0, 0);"">Executar...",<ul><li>Sólidos conhecimentos de programação e...,<ul><li>Elabora metodologias de cálculo para a...,<p>Vale refeição</p><p>Assistência Médica</p><...,Uberlândia,Minas Gerais
4,https://mutantbrvagas.gupy.io/job/eyJqb2JJZCI6...,Cientista de Dados Sênior,Não informado,Híbrido,Efetivo,Também p/ PcD,6986643.0,2024-04-05T17:52:22.645Z,2024-04-30,<p><strong>O QUE É SER UM MUTANT?</strong></p>...,<ul><li>Desenvolvimento em Python;</li><li>SQL...,<ul><li>Responsável pelo desenvolvimento de es...,<p><strong>O que você vai encontrar por aqui:<...,Osasco,São Paulo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,https://a3data.gupy.io/job/eyJqb2JJZCI6NDgwNzU...,Banco de Talentos - Pessoa Cientista de Dados ...,Não informado,Remoto,Banco de Talentos,Também p/ PcD,4807510.0,2023-05-15T20:07:19.851Z,2023-12-31,"<p class=""ql-align-justify""><span style=""color...","<p class=""ql-align-justify""><strong style=""col...","<p>P<span style=""color: black;"">ara ser uma </...","<p class=""ql-align-justify""><strong style=""col...",Belo Horizonte,Minas Gerais
68,https://slcmaquinas.gupy.io/job/eyJqb2JJZCI6ND...,Cientista de Dados Agro,Não informado,Presencial,Efetivo,Também p/ PcD,4456673.0,2023-03-15T17:16:23.137Z,2023-04-15,<p>Vem fazer parte de uma empresa Top Ser Huma...,"<ul><li class=""ql-align-justify""><span style=""...","<ul><li class=""ql-align-justify"">Coletar grand...",<p><strong>Benefícios:</strong></p><ul><li>Pla...,,None
69,https://vagasewave.gupy.io/job/eyJqb2JJZCI6Mzk...,Cientista de Dados,Não informado,Remoto,Efetivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,https://dexco.gupy.io/job/eyJqb2JJZCI6MTYwNzA3...,Banco de Talentos - Cientista de Dados,Não informado,Remoto,Banco de Talentos,Também p/ PcD,1607079.0,2022-03-17T20:58:41.000Z,2023-12-31,"<p class=""ql-align-justify""><span style=""color...",<p>• Desenvolvimento e expansão de nosso DataL...,<p>• Atuar como cientis de dados na evolução e...,<p>• Conhecimentos específicos e obrigatórios:...,São Paulo,São Paulo


In [39]:
import json

json_vaga = driver.find_element(By.XPATH,'//*[@id="__NEXT_DATA__"]').get_attribute('innerHTML')


json_vaga = json.loads(json_vaga)
json_vaga['props']['pageProps']['job']['id']
json_vaga['props']['pageProps']['job']['description']
json_vaga['props']['pageProps']['job']['publishedAt']
json_vaga['props']['pageProps']['job']['expiresAt']
json_vaga['props']['pageProps']['job']['description']
json_vaga['props']['pageProps']['job']['prerequisites']
json_vaga['props']['pageProps']['job']['responsibilities']
json_vaga['props']['pageProps']['job']['relevantExperiences']
json_vaga['props']['pageProps']['job']['addressCity']
json_vaga['props']['pageProps']['job']['addressState']

'<p><strong>O que te oferecemos:</strong></p><p><br></p><p><span style="color: rgba(0, 0, 0, 0.9);">🍛 </span><span style="color: black;">Vale Alimentação ou Vale Refeição;</span></p><p>👨🏼\u200d🎓 <span style="color: black;">Desconto em cursos, universidades e instituições de idiomas; </span></p><p><span style="color: rgba(0, 0, 0, 0.9);">📚 </span><span style="color: black;">Academia Stefanini - plataforma com cursos online, gratuitos, atualizados e com certificado; </span></p><p><span style="color: rgba(0, 0, 0, 0.9);">🗣 </span><span style="color: black;">Mentoring;</span></p><p>🏫 <span style="color: black;">Auxílio Creche; </span></p><p>💉 <span style="color: black;">Clube de vantagens para consultas e exames;</span></p><p>🏥 <span style="color: black;">Assistência Médica;</span></p><p>🦷<span style="color: black;">Assistência Odontológica; </span></p><p>💰 <span style="color: black;">Clube de vantagens e descontos nos melhores estabelecimentos;</span></p><p>👂🏾<span style="color: rgba(0, 0

In [36]:
def clean_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    clean_text = soup.get_text()
    return clean_text

clean_tags(json_vaga['props']['pageProps']['job']['description'])

'📢 Oportunidade para\xa0Cientista de Dados PL/SR!Home Office\ufeffEi, você já pensou em fazer parte de um time que só vem crescendo? 🚀Somos a maior multinacional brasileira e estamos entre as 100 maiores empresas de TI do mundo.Estamos presentes em 41 países com +30k colaboradores. A Stefanini tem como principal objetivo auxiliar os clientes a encontrar, por meio de soluções personalizadas, a maneira ideal para alcançar seus desafios, impulsionando a inovação digital.E queremos mais!🤩'